In [1]:
from copy import deepcopy
from collections import defaultdict
import random

## Reading and initializing functions

In [2]:
def get_states(filename, sudokusize=9):
    '''
    get_states is to read multiple sudokus in the format:
    ..8..4..5...64..5 etc.
    '''
    sudokus = []
    with open(filename) as sudoku_states:
        for state in sudoku_states:
            sudoku = []
            row = 1
            for i, content in enumerate(state):
                if i%sudokusize == 0 and i > 0:
                    row+=1
                try:
                    int(content)
                    clause = int(str(row) + str(i%sudokusize+1) + content)
                    sudoku.append(clause)
                except:
                    # is not convertible to int
                    continue
            sudokus.append(sudoku)
    return sudokus

def get_state_from_dimacs(filename, sudokusize=9):
    '''
    This function reads a sudoku state in dimacs format
    '''
    sudoku = []
    with open(filename) as state:
        for clause in state:
            sudoku.append(int(clause.split()[0]))
    return sudoku

def get_rules(filename):
    constraints = {} # dictionary with contraints as: clausenumber->list ->literals (true)
    pointers = {} # dictionary with pointers to clause numbers for literals as: literals->[clausenumbers]
    with open(filename) as sudoku_rules:
        for index, clause in enumerate(sudoku_rules):
            if index == 0:
                continue # skip first line
            constraints[index]=[]
            for literal in clause.split():
                lit = int(literal)
                if lit!=0:
                    constraints[index].append(lit)
                    if lit not in pointers.keys():
                        pointers[lit]=[index]
                    else:
                        pointers[lit].append(index)
                    
    return constraints, pointers

## DP helper functions for constraints updates

In [3]:
def assign_literal(constraints, pointers, literal):
    '''
    assign the given literal to be True.
    assignLiteral is aimed to be one recursion stack, so deepcopies are made
    in order to properly backtrack
    '''
    constraintsCopied = deepcopy(constraints)
    pointersCopied = deepcopy(pointers)
    constraintsUpdated, pointersUpdated = update_constraints(constraintsCopied, pointersCopied, literal)
    return constraintsUpdated, pointersUpdated
     

def update_constraints(constraints, pointers, literal):
    
    ## Find clauses with literal and remove them
    clauses = pointers[literal]
    for clause in clauses:
        try:
            del constraints[clause]
            pointers[literal].remove(clause)
        except:
            print("This exception should not occur, but it did")
            continue
            
    ## Find clauses with counter-literal and remove its counter-literals
    clauses = pointers[-literal]
    for clause in clauses:
        try:
            constraints[clause].remove(-literal)
            pointers[-literal].remove(clause)
        except:
            print("This exception should not occur either, but it did")
            continue
    
    return constraints, pointers
        
    
def check_tautologies(constraints, pointers):
    foundOne = False
    for clause in constraints.keys():
        literals = constraints[clause]
        counterLiterals = [-x for x in literals]
        for i in literals:
            for j in counterLiterals:
                if i == j:
                    del constraints[clause]
                    pointers[i].remove(clause)
                    foundOne = True
                    break
                  
    return constraints, pointers, foundOne  

## DP cases

In [16]:
def empty_clause(constraints):
    for clause in constraints.keys():
        if not constraints[clause]:
            return True
    return False


def unit_clause(constraints, pointers): 
    
    unitClauses = []
    
    for clause in constraints.keys():
        if len(constraints[clause])==1:
            unitClauses.append(clause)
    
    # or do this one step at a time in recursion instead of for loop?
    for u in unitClauses:
        literal = constraints[u][0]
        constraints, pointers = assign_literal(constraints, pointers, literal)
        
    if len(unitClauses) > 0: 
        return True, constraints
    else:
        return False, constraints

def pure_literal(constraints, pointers):
    foundOne = False
    literals = list(pointers.keys())
    counterLiterals = [-x for x in literals]
    pureLiterals = [x for x in literals if x not in counterLiterals]
    
    if len(pureLiterals)>0:
        foundOne=True
    
    # or do this one step at a time in recursion instead of for loop?
    for literal in pureLiterals:
        constraints, pointers = assign_literal(constraints, pointers, literal)

    return foundOne, constraints, pointers

def random_literal(constraints, pointers, prevLiteral):   
    if not prevLit:
        literal = random.choice(list(pointers.keys()))
    else:
        literal = -prevLiteral

    constraints, pointers = assign_literal(literal)
    
    return constraints, pointers, literal

## DP main function

In [13]:
def DP(constraints, pointers):
#     print(len(constraints))

    if len(constraints)==0:
        print("satisfiable solution")
        return True
   
    if empty_clause(constraints):
        print("no satisfiability")
        return False
   
    hasFound, constraints, pointers = unit_clause(constraints, pointers)
    if hasFound:
        DP(constraints,literalsDict)
        
    hasFound, constraints, pointers = pure_literal(constraints, pointers)
    if hasFound:
        DP(constraints, pointers)
                
    constraints, pointers, prevLiteral = random_literal(constraints, pointers, None)
    if DP(reducedConstraints, reducedPointers)==True:
        print("satisfiable solution")
        return True
    else:
        contraints, pointers, prevLiteral = random_literal(constraints, pointers, prevLiteral)
        DP(constraints, pointers)

In [17]:
def set_up():
    state = get_state_from_dimacs('sudoku-example.txt')
    print("Sudoku initial board state", state)
    constraints, pointers = get_rules('sudoku-rules.txt')
    print("Length of clauses:", len(constraints))
    ## Update contraints according to initial board state
    for literal in state:
        constraints, pointers = assign_literal(constraints, pointers, literal)
    print("clauses after board state:", len(constraints.keys()))
    constraints, pointers, foundOne = check_tautologies(constraints, pointers)
    if foundOne:
        print("Found tautologies and removed corresponding clauses")
    else:
        print("No tautologies found")
    print("Starting DP algorithm...")
    DP(constraints, pointers)
    
set_up()
    

Sudoku initial board state [168, 175, 225, 231, 318, 419, 444, 465, 493, 689, 692, 727, 732, 828, 886, 956, 961, 973]
Length of clauses: 11988
clauses after board state: 11952
No tautologies found
Starting DP algorithm...
This exception should not occur either, but it did
This exception should not occur either, but it did
This exception should not occur either, but it did
This exception should not occur either, but it did
This exception should not occur either, but it did
This exception should not occur either, but it did
This exception should not occur, but it did
This exception should not occur either, but it did
This exception should not occur, but it did
This exception should not occur either, but it did
This exception should not occur either, but it did
This exception should not occur either, but it did
This exception should not occur, but it did
This exception should not occur either, but it did
This exception should not occur, but it did
This exception should not occur either, b

KeyError: 3025